In [1]:
using LinearAlgebra

In [2]:
function gauss_jordan_elimination(A::Matrix, b::Union{Nothing, Vector, Matrix}=nothing; eptols = 1.0e-10)
    m, n = size(A)

    if b ≠ nothing
        @assert m == size(b)[1]
        B = [A b]
    else
        B = A
    end
    println(B)
    
    ld = 0 #선행 1 성분의 column index

    for i in 1:m
        termination = true # 종료 조건
        for j in (ld+1):n
            p = argmax(abs.(B[i:end, j])) + i -1
            
            if abs(B[p, j]) > eptols
                B[i,:], B[p, :] = B[p, :], B[i,:]
                ld = j
                termination = false
                break
            end
        end

        if termination 
            break
        end
        
        B[i, :] .= B[i, :]./B[i, ld]
        
        # 선행 1 성분의 열을 자신을 제외하고는 제거
        for k in 1:m
            if k ≠ i 
                B[k, :] .= B[k, :] .- (B[k, ld].* B[i, :])
            end
        end
    end

    if b ≠ nothing 
         return B[:, 1:n], B[:,(n+1):end]
    else 
         return B
    end
end

gauss_jordan_elimination (generic function with 2 methods)

In [3]:
L = [1.1 0. 0. 0.; 2. 2. 0. 0.; 3. 0.3 -3. 0.; 5. -2. 3. 7.]
b = one(L)
L2, x = gauss_jordan_elimination(L, b)
L*x .- b

[1.1 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 2.0 2.0 0.0 0.0 0.0 1.0 0.0 0.0; 3.0 0.3 -3.0 0.0 0.0 0.0 1.0 0.0; 5.0 -2.0 3.0 7.0 0.0 0.0 0.0 1.0]


4×4 Matrix{Float64}:
 -1.11022e-16   3.05311e-17   0.0           3.05311e-17
  0.0           0.0          -5.55112e-17  -2.77556e-17
  1.11022e-16  -9.71445e-17   0.0          -9.57567e-17
 -6.66134e-16   2.35922e-16  -5.55112e-17   2.22045e-16

In [4]:
L*x

4×4 Matrix{Float64}:
  1.0           3.05311e-17   0.0           3.05311e-17
  0.0           1.0          -5.55112e-17  -2.77556e-17
  1.11022e-16  -9.71445e-17   1.0          -9.57567e-17
 -6.66134e-16   2.35922e-16  -5.55112e-17   1.0

In [44]:
function PLU(A::Matrix; eptols = 1.0e-10)
    m, n = size(A)
    @assert m == n

    L, P, U = one(A), one(A), zero(A)
    B = copy(A)
    
    for i in 1:(m-1)
        # p 는 i 행과 피보팅을 하는 행벡터이다.
        p = findmax(abs.(B[i:end, i]))[2] + i -1
        
        if abs(B[p, i]) < eptols
            error("Singularity error")    
        end

        # 행교환 행렬 P 와 계산하는 행렬 B 에 대한 피보팅
        P[i,:], P[p, :] = P[p, :], P[i, :]
        B[i,:], B[p, :] = B[p, :], B[i, :]
        
        if i>1
            L[i,1:i-1], L[p, 1:i-1] = L[p, 1:i-1], L[i, 1:i-1]
        end
        
        U[i, i] = B[i, i]
        U[i, (i+1):end] = B[i, (i+1):end]
        L[(i+1):end, i] = B[(i+1):end, i] / B[i, i]
        B[(i+1):end, (i+1):end] = B[(i+1):end, (i+1):end] - (L[(i+1):end, i:i] * U[i:i, (i+1):end])
    end
    U[m, m] = B[m, m]
    return P, L, U
end

function PLU2(A::Matrix; eptols = 1.0e-10)
    m, n = size(A)
    M1, M2 = minmax(m, n)

    II = Float64.(Matrix(I, M2, M2))
    L = II[1:m, 1:n]
    P = II[1:m, 1:m]
    U = zeros(n, n)

    B = copy(A)
    for i in 1:(M1-1)
        # p 는 i 행과 피보팅을 하는 행벡터이다.
        p = findmax(abs.(B[i:end, i]))[2] + i -1
   
        if abs(B[p, i]) < eptols
            error("Singularity error")    
        end

        # 행교환 행렬 P 와 계산하는 행렬 B 에 대한 피보팅
        P[i,:], P[p, :] = P[p, :], P[i, :]
        B[i,:], B[p, :] = B[p, :], B[i, :]
        
        if i>1
            L[i,1:i-1], L[p, 1:i-1] = L[p, 1:i-1], L[i, 1:i-1]
        end
        
        U[i, i] = B[i, i]
        U[i, (i+1):end] = B[i, (i+1):end]
        L[(i+1):end, i] = B[(i+1):end, i] / B[i, i]
        B[(i+1):end, (i+1):end] = B[(i+1):end, (i+1):end] - (L[(i+1):end, i:i] * U[i:i, (i+1):end])
    end
    #U[m, m] = B[m, m]
    return P, L, U
end

PLU2 (generic function with 1 method)

In [45]:
A = 3.0*I + rand(5, 5)

5×5 Matrix{Float64}:
 3.38911   0.98166    0.565677  0.482919  0.640385
 0.383832  3.94393    0.987403  0.230645  0.397523
 0.913796  0.157428   3.259     0.92937   0.927977
 0.78926   0.806854   0.67686   3.18598   0.193107
 0.293189  0.0649724  0.996866  0.532894  3.33063

In [46]:
P, L, U=PLU(A)

([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.1132545390117409 1.0 … 0.0 0.0; … ; 0.23288153368151512 0.15086918126469565 … 1.0 0.0; 0.0865093006455428 -0.00520523256881569 … 0.08408985717393506 1.0], [3.389106146919485 0.9816604292842102 … 0.4829187307504622 0.6403854643155922; 0.0 3.832748773453186 … 0.17595240976125126 0.32499595828884476; … ; 0.0 0.0 … 2.942795546491407 -0.1040949220234881; 0.0 0.0 … 0.0 3.053175578874737])

In [47]:
P2, L2, U2 = PLU2(A)

([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [1.0 0.0 … 0.0 0.0; 0.1132545390117409 1.0 … 0.0 0.0; … ; 0.23288153368151512 0.15086918126469565 … 1.0 0.0; 0.0865093006455428 -0.00520523256881569 … 0.08408985717393506 1.0], [3.389106146919485 0.9816604292842102 … 0.4829187307504622 0.6403854643155922; 0.0 3.832748773453186 … 0.17595240976125126 0.32499595828884476; … ; 0.0 0.0 … 2.942795546491407 -0.1040949220234881; 0.0 0.0 … 0.0 0.0])

In [48]:
P2, L2, U2 = PLU2(A[1:4, 1:3])

([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [1.0 0.0 0.0; 0.1132545390117409 1.0 0.0; 0.26962750344329345 -0.027983692365789455 1.0; 0.23288153368151512 0.15086918126469565 0.0], [3.389106146919485 0.9816604292842102 0.5656771574877147; 0.0 3.832748773453186 0.9233372361859908; 0.0 0.0 0.0])

In [53]:
A[1:4, 1:3] .- L2*U2

4×3 Matrix{Float64}:
  0.0          0.0  0.0
  0.0          0.0  0.0
 -1.11022e-16  0.0  3.13232
  0.0          0.0  0.405821

In [15]:
lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
5×5 Matrix{Float64}:
 1.0         0.0       0.0        0.0        0.0
 0.181338    1.0       0.0        0.0        0.0
 0.00574423  0.123859  1.0        0.0        0.0
 0.0134574   0.235726  0.0644889  1.0        0.0
 0.0322485   0.233109  0.0861488  0.0581758  1.0
U factor:
5×5 Matrix{Float64}:
 3.79574  0.209682  0.604866  0.00203435   0.134018
 0.0      3.51047   0.350753  0.0271711   -0.0149451
 0.0      0.0       3.03858   0.559769     0.431379
 0.0      0.0       0.0       3.17808      0.427015
 0.0      0.0       0.0       0.0          3.92945